In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Vou usar a base de dados fornecida com as classes Cachorros, Gatos e Pandas para fazer uma rede convolucional do zero usando TensorFlow. Depois será feito uma comparação com uma rede pré-treinada para avaliar as diferenças entre os dois.

In [ ]:
import splitfolders
import tensorflow as tf
from tensorflow.keras import layers, models

Foi necessário criar um diretorio origem e um diretório destino para que fosse feito a divisão entre os dados que seriam usados para o treinamento e os de teste.

In [ ]:
diretorio_origem = 'animals'
diretorio_destino = 'animalsd'

splitfolders.ratio(diretorio_origem, output=diretorio_destino, seed=1337, ratio=(0.8,0.2))

Para fazer a divisão entre os dados foi usado a função splitfolders que cria uma pasta train e uma val. Train é a de treinamento e val é a de validação (teste) da rede neural convolucional.

Em seguida são os códigos usados para fazer a CNN.

In [ ]:
diretorio_treinamento = 'animalsd\\train'
diretorio_teste = 'animalsd\\val'

gerador_treinamento = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

dados_treinamento = gerador_treinamento.flow_from_directory(diretorio_treinamento,target_size=(224,224), batch_size = 32, class_mode='categorical')

gerador_teste = ImageDataGenerator(rescale = 1.0/225)

dados_teste = gerador_teste.flow_from_directory(diretorio_teste, target_size=(224,224), batch_size=32,class_mode='categorical')

In [ ]:
modelo = models.Sequential([layers.Conv2D(32,(3,3),activation='relu', input_shape=(224,224,3)),layers.MaxPooling2D((2,2)), layers.Conv2D(64,(3,3),activation='relu'),layers.MaxPooling2D((2,2)),layers.Conv2D(128,(3,3),activation='relu'),layers.MaxPooling2D((2,2)),layers.Flatten(),layers.Dense(128, activation='relu'),layers.Dense(3, activation='softmax')])

In [ ]:
modelo.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import scipy
from scipy import stats

In [ ]:
modelo.fit(dados_treinamento, epochs=10, validation_data = dados_teste)

In [ ]:
resultados = modelo.evaluate(dados_teste)
print(f'Acurácia :{resultados[1]*100:.2f}%')

A acurácia do modelo utilizado deu em torno de 70%. Acredito que seja razoável mas poderíamos melhorar isso fazendo ajustando alguns hiperparâmetros, a taxa de aprendizado, o número de épocas.

Algo que poderia melhorar os resultados seriam fazer algumas alterações no código de aprendizagem como uma rotação aleatória de até 20 graus, deslocamento horizontal e vertical aleatório, corte lateral e zoom aleatório.

In [ ]:
from tensorflow.keras.applications import VGG16

Aqui será feito o uso da base de dados pré-treinada.

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
num_classes = 3
output_layer= tf.keras.layers.Dense(num_classes, activation ='softmax')(base_model.output)
model = tf.keras.Model(inputs=base_model.input, outputs=output_layer)

model.compile(optimizer='adam',loss= 'categorical_crossentropy', metrics=['accuracy'])

model.fit(diretorio_treinamento, epochs=10, validation_data=diretorio_teste)

test_loss, test_acc = model.evaluate(diretorio_teste)
print(f'Acurácia no conjunto de teste: {test_acc * 100:.2f}%')

NLP


In [ ]:
import spacy
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords



In [ ]:
nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')


def preprocess_text(text):
    text  = text.lower()

    text = re.sub(r'[^a-zA-ZO-9/s]','',text)

    doc = nlp(text)

    tokens = [token.text for token in doc if token.text not in stop_words]